![manufacturing gears](manufacturing.jpg)

Manufacturing processes for any product is like putting together a puzzle. Products are pieced together step by step, and keeping a close eye on the process is important.

For this project, you're supporting a team that wants to improve how they monitor and control a manufacturing process. The goal is to implement a more methodical approach known as statistical process control (SPC). SPC is an established strategy that uses data to determine whether the process works well. Processes are only adjusted if measurements fall outside of an acceptable range. 

This acceptable range is defined by an upper control limit (UCL) and a lower control limit (LCL), the formulas for which are:

$ucl = avg\_height + 3 * \frac{stddev\_height}{\sqrt{5}}$

$lcl = avg\_height - 3 * \frac{stddev\_height}{\sqrt{5}}$

The UCL defines the highest acceptable height for the parts, while the LCL defines the lowest acceptable height for the parts. Ideally, parts should fall between the two limits.

Using SQL window functions and nested queries, you'll analyze historical manufacturing data to define this acceptable range and identify any points in the process that fall outside of the range and therefore require adjustments. This will ensure a smooth running manufacturing process consistently making high-quality products.

## The data
The data is available in the `manufacturing_parts` table which has the following fields:
- `item_no`: the item number
- `length`: the length of the item made
- `width`: the width of the item made
- `height`: the height of the item made
- `operator`: the operating machine

In [12]:
-- Corrected SQL query
WITH temp AS (
SELECT item_no, length, width, height, operator,
    AVG(height) OVER (PARTITION BY operator ORDER BY operator ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS avg_height,
    STDDEV(height) OVER (PARTITION BY operator ORDER BY operator ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS stddev_height,
    ROW_NUMBER() OVER(PARTITION BY operator ORDER BY operator) AS row_number
FROM manufacturing_parts
), calc AS (
SELECT *, 
(avg_height + (3 * stddev_height/SQRT(5))) AS ucl,
(avg_height - (3 * stddev_height/SQRT(5))) AS lcl
FROM temp
)
SELECT calc.operator, calc.row_number, calc.height, calc.avg_height, calc.stddev_height, calc.ucl, calc.lcl,
(CASE WHEN height > lcl AND height < ucl THEN False ELSE TRUE END) As alert
FROM calc
WHERE row_number >= 5;

,operator,row_number,height,avg_height,stddev_height,ucl,lcl,alert
0,Op-1,5,19.46,19.778,1.062812,21.203912,18.352088,False
1,Op-1,6,20.36,19.912,1.090812,21.375477,18.448523,False
2,Op-1,7,20.22,20.030,1.084574,21.485108,18.574892,False
3,Op-1,8,21.03,19.934,0.931225,21.183369,18.684631,False
4,Op-1,9,19.78,20.170,0.598832,20.973418,19.366582,False
...,...,...,...,...,...,...,...,...
415,Op-9,31,19.93,19.898,0.810660,20.985615,18.810385,False
416,Op-9,32,20.71,19.910,0.824894,21.016711,18.803289,False
417,Op-9,33,20.00,20.154,0.537429,20.875037,19.432963,False
418,Op-9,34,21.33,20.284,0.737753,21.273800,19.294200,True
